# Check CFHT Zero Point Value

## Theoretical Background

The Zero Point value (in ADU/sec) for the `r` filter in the [CFHT specifications page](https://www.cfht.hawaii.edu/Instruments/Imaging/Megacam/generalinformation.html) is 26.22 and it seems to be over estimated. In the [LSST GitHub code](https://github.com/LSSTDESC/WeakLensingDeblending/blob/9f851f79f6f820f815528d11acabf64083b6e111/descwl/survey.py#L288) it is recomputed and its new value is 10.72.

In the following we are going to recompute the Zero Point value using the formulas in the [LSST GitHub issue #1](https://github.com/LSSTDESC/WeakLensingDeblending/issues/1).

We have:

\begin{equation}
z_p = m_0+2.5 \log_{10}(s_0[R,m_0]) \quad,
\end{equation}

where $z_p$ is the Zero Point AB magnitude, $m_0$ is a fiducial source of AB magnitude of value 24 and $s_0[R,m_0]$ the corresponding flux, such that:

\begin{equation}
s_0[R,m_0]=s[f_{AB},R]\cdot 10^{-0.4m_0} \quad,
\end{equation}

with $s[f_{AB},R]$ is defined as:

\begin{equation}
s[f_{AB},R]=A \int_0^{+\infty}f(\lambda)\cdot\frac{\lambda}{hc}\cdot R(\lambda) d\lambda \quad ,
\end{equation}

here $A=8.022$ is telescope effective area in $m^2$, $f(\lambda)=f_{AB}(\lambda)=1.08855 \cdot 10^{-5} (J/s/m^2/nm)$, $h$ is the Planck constant, $c$ is the speed of light and $\lambda$ (in nm) and $R(\lambda)$ value are given in the R.MP9602 third generation [filter response table](http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/megapipe/docs/filt.html).

## Numerical Application

First, extract and convert $\lambda$ and $R(\lambda)$ values, load other constants and compute $s[f_{AB},R]$.
> In the table, $\lambda$ values are in Angstroms and should be converted to nm.

In [1]:
import numpy as np
from scipy.constants import h,c

file_name = 'r_lambda.txt'
lamb, r = np.loadtxt(file_name, unpack=True)
lamb /= 10 #convert from Angstrom to nm
m0 = 24
f = 1.08855/(lamb**2) * 10 ** (-5) # J/s/m^2/nm^-1
A = 8.022 #m^2

# use the  trapezoidal rule to perform a numerical integration
sAB = A * np.trapz(f*lamb*r/(h*c), lamb)

print(r'The value of s[f_AB,R] is {}'.format(np.format_float_scientific(sAB)))

The value of s[f_AB,R] is 5.155817453958427e+19


Deduce the values of $s_0[R,m_0]$ then $z_p$.

In [2]:
s0 = sAB * 10 ** (-0.4*m0)
zp = m0 + 2.5*np.log10(s0)
print(r'The value of s_0[R, m_0] is {}'.format(np.format_float_scientific(s0)))
print(r'The magnitude of the zero point is {}'.format(np.format_float_scientific(zp)))

The value of s_0[R, m_0] is 1.2950827905938398e+10
The magnitude of the zero point is 4.928074383098289e+01
